In [ ]:
#
# Import Packages Required for this Notebook
#
from pyspark.sql import SparkSession

print("Successfully imported all packages for this notebook.")

In [ ]:
#
# Configure run-time parameters for this notebook
#
transient_tables = True

layer = "bronze"
db_schema = "dbo"
application = "warehouse"
workspace_id = "3ac7ce42-ae74-4e7d-8ac3-5ce8358a30df" ## Adv Wrks DE 3 Dev
lakehouse_id = "50402dac-ce50-4831-af2b-7d65ca8fe7db" ## AdventureWorks_Lakehouse

print("Successfully configured all paramaters for this run.")
print(f"transient_tables = {transient_tables}")

In [ ]:
#
# Define the OneLake folder path
#
folder = "/Files/" + layer + "/" + application
folder_path = "abfss://" + workspace_id + "@onelake.dfs.fabric.microsoft.com/" + lakehouse_id + folder

print(f"Configured to process from '{layer}' layer files in:\n{folder_path}")

In [ ]:
#
# Create the Spark session
#
app_name = "DropLakehouseTables"

# Get the current Spark session
spark = SparkSession.builder \
    .appName(app_name) \
    .getOrCreate()

print(f"Spark session {app_name} has been created successfully.")

In [ ]:
#
# Drop the Lakehouse tables
#
file_list = spark.read.format("binaryFile").load(folder_path).select("path").collect()

# Drop all tables to to associated CSV data
for file in sorted(file_list):
    file_path = file["path"]
    if file_path.endswith(".csv"):
        # Extract the table name from the file name
        table_name = file_path.split("/")[-1].split(".")[0]
        
        # Drop the table
        if transient_tables:
            full_table_name = db_schema + "." + "slv_" + table_name
        else:
            full_table_name = db_schema + "." + table_name
        
        spark.sql(f"DROP TABLE IF EXISTS {full_table_name}")
        print(f"Dropped table {full_table_name}")

In [ ]:
#
# Stop the Spark session
# NOTE: frees up limited F2 SKU capacity resources
#
spark.stop()

print("Spark session has been stopped successfully.")